In [1]:
# Data available: https://www.ncdc.noaa.gov/cdo-web/search
# 1.csv Jan 1, 2013- April 5, 2022
# 2.csv Jan 1, 2003 – December 31, 2012 
# 3.csv Jan 1, 1993 - December 31, 2002 
# 4.csv Jan 1, 1983 - December 31, 1992 
# 5.csv Jan 1, 1973 - December 31, 1982 

In [2]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [3]:
import sqlite3
conn = sqlite3.connect('database.db')
cur = conn.cursor()

In [4]:
ncd_cols = ['DATE', 
            'HourlyAltimeterSetting', 'HourlyDryBulbTemperature', 
            'HourlyDewPointTemperature', 'HourlyRelativeHumidity', 
            'HourlyWindSpeed', 'HourlyWindDirection']

In [5]:
df1 = pd.read_csv('datasets/nenana_climate/1.csv', usecols = ncd_cols) 
df2 = pd.read_csv('datasets/nenana_climate/2.csv', usecols = ncd_cols) 
df3 = pd.read_csv('datasets/nenana_climate/3.csv', usecols = ncd_cols) 
df4 = pd.read_csv('datasets/nenana_climate/4.csv', usecols = ncd_cols) 
df5 = pd.read_csv('datasets/nenana_climate/5.csv', usecols = ncd_cols)  

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (42,43,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (42,43,44,49,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (42,43,44,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_as

In [6]:
date_object = lambda d : datetime.strptime(d, '%Y-%m-%dT%H:%M:%S')
shortened_date = lambda d : datetime.strptime(d[:10], '%Y-%m-%d')
# to fix some numerical values saved as strings; some end with 's'
remove_s = lambda n : (float(n[:-1]) if n.endswith('s') else float(n)) if type(n)== str else n

In [7]:
df1 = df1.dropna()
df1['date_obj']= df1['DATE'].apply(date_object)
df1['short_day']= df1['DATE'].apply(shortened_date)
df1['HourlyDewPointTemperature'] = df1['HourlyDewPointTemperature'].apply(remove_s)
df1['HourlyAltimeterSetting']   = df1['HourlyAltimeterSetting'].apply(remove_s)
df1['HourlyDryBulbTemperature'] = df1['HourlyDryBulbTemperature'].apply(remove_s)

In [8]:
df2 = df2.dropna()
df2['date_obj']= df2['DATE'].apply(date_object)
df2['short_day']= df2['DATE'].apply(shortened_date)
# this dataset has some random astericks for values 
df2 = df2.replace('*', pd.NA) 
df2 = df2.dropna()
df2['HourlyDewPointTemperature'] = df2['HourlyDewPointTemperature'].apply(remove_s)
df2['HourlyAltimeterSetting']   = df2['HourlyAltimeterSetting'].apply(remove_s)
df2['HourlyDryBulbTemperature'] = df2['HourlyDryBulbTemperature'].apply(remove_s)

In [9]:
df3 = df3.dropna()
df3['date_obj']= df3['DATE'].apply(date_object)
df3['short_day']= df3['DATE'].apply(shortened_date)
df3['HourlyDewPointTemperature'] = df3['HourlyDewPointTemperature'].apply(remove_s)
df3['HourlyAltimeterSetting']   = df3['HourlyAltimeterSetting'].apply(remove_s)
df3['HourlyDryBulbTemperature'] = df3['HourlyDryBulbTemperature'].apply(remove_s)

In [10]:
df4 = df4.dropna()
df4['date_obj']= df4['DATE'].apply(date_object)
df4['short_day']= df4['DATE'].apply(shortened_date)
df4['HourlyDewPointTemperature'] = df4['HourlyDewPointTemperature'].apply(remove_s)
df4['HourlyAltimeterSetting']   = df4['HourlyAltimeterSetting'].apply(remove_s)
df4['HourlyDryBulbTemperature'] = df4['HourlyDryBulbTemperature'].apply(remove_s)

In [11]:
df5 = df5.dropna()
df5['date_obj']= df5['DATE'].apply(date_object)
df5['short_day']= df5['DATE'].apply(shortened_date)
df5['HourlyDewPointTemperature'] = df5['HourlyDewPointTemperature'].apply(remove_s)
df5['HourlyAltimeterSetting']   = df5['HourlyAltimeterSetting'].apply(remove_s)
df5['HourlyDryBulbTemperature'] = df5['HourlyDryBulbTemperature'].apply(remove_s)

In [12]:
dailies = pd.concat([df1.groupby('short_day').mean(), 
        df2.groupby('short_day').mean(), df3.groupby('short_day').mean(), 
        df4.groupby('short_day').mean(), df5.groupby('short_day').mean()])

In [13]:
dailies.index.min(), dailies.index.max()

(Timestamp('1973-01-05 00:00:00'), Timestamp('2022-01-01 00:00:00'))

In [14]:
dailies['Year'] = dailies.index.year

In [15]:
dailies = dailies.reset_index()

In [16]:
dailies.to_sql('daily_climate', conn, if_exists='replace', index = False)

In [17]:
dailies

,short_day,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyWindSpeed,HourlyWindDirection,Year
0,2013-01-01,29.340833,16.333333,30.625000,55.625000,NaN,NaN,2013
1,2013-01-02,29.359167,2.958333,8.416667,78.625000,NaN,NaN,2013
2,2013-01-03,29.560417,13.375000,27.916667,54.541667,NaN,NaN,2013
3,2013-01-04,29.714167,2.375000,10.958333,69.375000,NaN,NaN,2013
4,2013-01-05,29.543333,4.875000,16.416667,60.958333,NaN,NaN,2013
...,...,...,...,...,...,...,...,...
15032,1982-12-27,30.000000,5.333333,14.000000,69.000000,11.333333,166.666667,1982
15033,1982-12-28,30.000000,13.000000,18.000000,81.000000,89.000000,330.000000,1982
15034,1982-12-29,30.000000,24.400000,39.000000,57.000000,3.600000,125.000000,1982
15035,1982-12-30,30.174583,27.250000,30.166667,89.333333,3.166667,152.500000,1982
